In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from bqplot import pyplot 

import folium
import fiona
import gdal
import rasterio
import shapefile as shp

import os

import chart_studio.plotly
import plotly.graph_objs as go
# import plotly

chart_studio.tools.set_credentials_file(username='dishha', api_key='CmkdrDbPF7sMOY60fOMY')
mapbox_token = os.getenv('pk.eyJ1IjoiZGlzaGhhIiwiYSI6ImNrMDlmNGlzczA2NzQzZG9jcjc5NjEwZnoifQ.kH_uVGITVBICzi6QnWcVhg')

from mapboxgl.utils import create_color_stops, df_to_geojson
from mapboxgl.viz import CircleViz

from opencage.geocoder import OpenCageGeocode
key = 'cddaa05552b243d390c990b52dbd72dd' #opencage geocode key



In [3]:
df = pd.read_excel("All_Cities.xlsx")

In [4]:
df.head()

,year,month,day,SO2,NO2,PM10,Stn Code,State,City
0,2015,1,1,9.0,18.0,46.0,233,Andhra Pradesh,Visakhapatnam
1,2015,1,4,8.0,18.0,27.0,233,Andhra Pradesh,Visakhapatnam
2,2015,1,7,9.0,15.0,29.0,233,Andhra Pradesh,Visakhapatnam
3,2015,1,10,7.0,17.0,78.0,233,Andhra Pradesh,Visakhapatnam
4,2015,1,13,9.0,23.0,35.0,233,Andhra Pradesh,Visakhapatnam


In [10]:
df.shape

(213083, 9)

In [4]:
df["year"].unique()

array([2015, 2014, 2013, 2012, 2011], dtype=int64)

In [5]:
df["State"].unique()

array(['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli', 'Daman and Diu', 'Delhi',
       'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu & Kashmir',
       'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh',
       'Maharashtra', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha',
       'Puducherry', 'Punjab', 'Rajasthan', 'Tamil Nadu', 'Telangana',
       'Uttarakhand', 'Uttar Pradesh', 'West Bengal', 'Bihar'],
      dtype=object)

In [8]:
df["State"].nunique()

31

In [7]:
df["City"].unique()

array(['Visakhapatnam', 'Tirupati', 'Vijayawada', 'Kurnool', 'Kakinada',
       'Nellore', 'Chittoor', 'Guntur', 'Ananthapur', 'Kadapa', 'Eluru',
       'Srikakulam', 'Vizianagaram', 'Ongole', 'Rajahmundry', 'Itanagar',
       'Naharlagun', 'Guwahati', 'Bongaigaon', 'Tezpur', 'Sivasagar',
       'Dibrugarh', 'Golaghat', 'Daranga', 'Silcher', 'Margherita',
       'Lakhimpur', 'Tinsukia', 'Nagaon', 'Nalbari', 'Chandigarh',
       'Bhilai', 'Raipur', 'Korba', 'Bilaspur', 'Khadoli', 'Daman',
       'Delhi', 'Vasco', 'Panaji', 'Curchorem', 'Usgao', 'Codli', 'Honda',
       'Bicholim', 'Amona', 'Assanora', 'Margao', 'Mapusa', 'Ponda',
       'Sanguem', 'Tilamol', 'Kundaim', 'Cuncolim', 'Surat', 'Vadodara',
       'Ahmedabad', 'Vapi', 'Ankleshwar', 'Rajkot', 'Jamnagar',
       'ANKLESHWAR', 'VAPI', 'Bharuch', 'Bhuj', 'MORBI', 'SANAND',
       'Sarigam', 'Faridabad', 'Shimla', 'Paonta Sahib', 'Parwanoo',
       'Damtal', 'Baddi', 'Kala Amb', 'Nalagarh', 'Sunder Nagar', 'Una',
       'Manali', 

In [9]:
df["City"].nunique()

264

Adding Lat Longitude Coordinates Columns

In [7]:
geocoder = OpenCageGeocode(key)

In [8]:
query = 'Andhra Pradesh, Visakhapatnam'  
results = geocoder.geocode(query)
print (results)

[{'annotations': {'DMS': {'lat': "17° 43' 23.25936'' N", 'lng': "83° 18' 4.62312'' E"}, 'MGRS': '44QQE4405161046', 'Maidenhead': 'NK17pr63dn', 'Mercator': {'x': 9273056.54, 'y': 1992165.796}, 'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?node=245641840#map=16/17.72313/83.30128', 'url': 'https://www.openstreetmap.org/?mlat=17.72313&mlon=83.30128#map=16/17.72313/83.30128'}, 'UN_M49': {'regions': {'ASIA': '142', 'IN': '356', 'SOUTHERN_ASIA': '034', 'WORLD': '001'}, 'statistical_groupings': ['LEDC']}, 'callingcode': 91, 'currency': {'alternate_symbols': ['Rs', '৳', '૱', '௹', 'रु', '₨'], 'decimal_mark': '.', 'html_entity': '&#x20b9;', 'iso_code': 'INR', 'iso_numeric': '356', 'name': 'Indian Rupee', 'smallest_denomination': 50, 'subunit': 'Paisa', 'subunit_to_unit': 100, 'symbol': '₹', 'symbol_first': 1, 'thousands_separator': ','}, 'flag': '🇮🇳', 'geohash': 'tg5hzd6hsw7qpn6myx87', 'qibla': 282.52, 'roadinfo': {'drive_on': 'left', 'speed_in': 'km/h'}, 'sun': {'rise': {'apparent': 15

In [9]:
lat = results[0]['geometry']['lat']

lng = results[0]['geometry']['lng']

print (lat, lng)

17.7231276 83.3012842


In [12]:
list_lat = []   # create empty lists
list_long = []
results_dict = {}

In [13]:
for index, row in df.iterrows():
    City = row['City']
    State = row['State']
    query = str(City)+','+str(State)
    if query not in results_dict:
        results_dict[query] = geocoder.geocode(query)

In [14]:
for index, row in df.iterrows():
    City = row['City']
    State = row['State']
    query = str(City)+','+str(State)
    for k in results_dict:
        if query == k:
            lat = results_dict[k][0]['geometry']['lat']
            long = results_dict[k][0]['geometry']['lng']
            list_lat.append(lat)
            list_long.append(long)

In [15]:
df['lat'] = list_lat   
df['lon'] = list_long

In [20]:
df.head()

,year,month,day,SO2,NO2,PM10,Stn Code,State,City,lat,lon
0,2015,1,1,9.0,18.0,46.0,233,Andhra Pradesh,Visakhapatnam,17.723128,83.301284
1,2015,1,4,8.0,18.0,27.0,233,Andhra Pradesh,Visakhapatnam,17.723128,83.301284
2,2015,1,7,9.0,15.0,29.0,233,Andhra Pradesh,Visakhapatnam,17.723128,83.301284
3,2015,1,10,7.0,17.0,78.0,233,Andhra Pradesh,Visakhapatnam,17.723128,83.301284
4,2015,1,13,9.0,23.0,35.0,233,Andhra Pradesh,Visakhapatnam,17.723128,83.301284


In [ ]:
for index, row in df.iterrows():
    Date = pd.to_datetime()

In [22]:
df["Date"] = pd.to_datetime((df.year*10000+df.month*100+df.day).apply(str),format='%Y%m%d')

In [23]:
df.head()

,year,month,day,SO2,NO2,PM10,Stn Code,State,City,lat,lon,Date
0,2015,1,1,9.0,18.0,46.0,233,Andhra Pradesh,Visakhapatnam,17.723128,83.301284,2015-01-01
1,2015,1,4,8.0,18.0,27.0,233,Andhra Pradesh,Visakhapatnam,17.723128,83.301284,2015-01-04
2,2015,1,7,9.0,15.0,29.0,233,Andhra Pradesh,Visakhapatnam,17.723128,83.301284,2015-01-07
3,2015,1,10,7.0,17.0,78.0,233,Andhra Pradesh,Visakhapatnam,17.723128,83.301284,2015-01-10
4,2015,1,13,9.0,23.0,35.0,233,Andhra Pradesh,Visakhapatnam,17.723128,83.301284,2015-01-13


In [25]:
df.to_csv('updated.csv')